In [ ]:
import base
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rushd as rd
import scipy as sp
import seaborn as sns
from statannotations.Annotator import Annotator

# enables concurrent editing of base.py
from importlib import reload
reload(base)

### Load data

Load lenti data for all cell types (`data`)

In [ ]:
base_path = rd.datadir/'instruments'/'data'/'attune'
metadata_path = rd.datadir/'projects'/'miR-iFFL'/'plasmids'
data, quantiles, stats, metadata = base.load_data(base_path, metadata_path, 'lenti')

Load application-relevant transfection data (`data2`)

In [ ]:
data2, quantiles2, stats2, metadata2 = base.load_data(base_path, metadata_path, 'application')

Load qPCR data (`data_qpcr`)

In [ ]:
data_qpcr, stats_qpcr, _ = base.load_data(base_path, metadata_path, 'qPCR')

In [ ]:
# Create dicts to specify colors/markers
metadata_dict = metadata.set_index('construct').to_dict('dict')
main_palette = metadata_dict['color']
main_markers = metadata_dict['markers']
main_markers['UI'] = 'D'

# Create dicts to specify colors/markers
metadata2 = base.get_metadata(metadata_path/'construct-metadata.xlsx', 'applications')
metadata_dict2 = metadata2.set_index('construct').to_dict('dict')
app_palette = metadata_dict2['color']
app_markers = metadata_dict2['markers']

In [ ]:
# Create palette for -/+ dox conditions (qPCR)
pal_dox = {k+'_True': v for k,v in metadata_dict2['color'].items()}
pal_none = {k+'_False': base.get_light_color(v) for k,v in metadata_dict2['color'].items()}
condition_palette = pal_dox | pal_none

In [ ]:
# Since there is no marker-only condition, save the output expression stats for untransduced cells
baseline_df = data[(data['construct'].isin(['UI','UT'])) | (data['virus_dilution']==0)].groupby(['cell','dox','exp','biorep'])['output'].apply(sp.stats.gmean).rename('output_gmean').reset_index()

### Set up figure

In [ ]:
# Set plotting context
sns.set_style('ticks')
sns.set_context('paper', font_scale=1.0, rc=base.rc_context)
plt.rcParams.update(base.rc_params)
scatter_kwargs = base.scatter_kwargs
annotate_kwargs = base.annotate_kwargs

# Create the overall figure, gridspec, and add subfigure labels
fig = plt.figure(figsize=(base.figure_width['full'], 1.5*6))
fig_gridspec = matplotlib.gridspec.GridSpec(6, 6, figure=fig, wspace=0.4, hspace=0.4, width_ratios=[1,1,1,0.6,0.8,1.6],)

subfigures = {
    'A': fig.add_subfigure(fig_gridspec[0,:3]),
    'B': fig.add_subfigure(fig_gridspec[0,3:]),
    'C': fig.add_subfigure(fig_gridspec[1,:5]),
    'D': fig.add_subfigure(fig_gridspec[2,:3]),
    'E': fig.add_subfigure(fig_gridspec[2,3:]),
    'F': fig.add_subfigure(fig_gridspec[3,:4]),
    'G': fig.add_subfigure(fig_gridspec[3,4]),
    'H': fig.add_subfigure(fig_gridspec[4,:4]),
    'I': fig.add_subfigure(fig_gridspec[4,4]),
    'J': fig.add_subfigure(fig_gridspec[5,:]),
    'K': fig.add_subfigure(fig_gridspec[3,5]),
    'L': fig.add_subfigure(fig_gridspec[4,5]),
}

# Add subpanel labels
for label, subfig in subfigures.items():
    if '2' in label: continue
    subfig.add_artist(matplotlib.text.Text(x=0, y=1, text=f'{label}', fontsize=base.font_sizes['subpanel_label'], 
                                           fontweight='bold', verticalalignment='top',transform=subfig.transSubfigure))

# Save to output folder
output_path = rd.rootdir/'output'/'fig_applications-supp'/'fig_applications-supp.pdf'
fig.savefig(rd.outfile(output_path))

In [ ]:
subfig = subfigures['A']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.4, bottom=0.3, top=0.6, right=0.15)
axes = subfig.subplots(1,3, gridspec_kw=dict(wspace=0.5))

plot_df = stats2[stats2['group'].isin(['controller','base']) & (stats2['design']<=1) & (stats2['promoter']=='EF1a') & 
                 (stats2['data_group']=='iPS11_transfection')]

# mean
ax = axes[0]
for construct, group in plot_df.groupby('construct'):
    sns.pointplot(data=group, x='ts_label', y='output_gmean', hue='construct', palette=app_palette,
                  ax=ax, marker=app_markers[construct], **scatter_kwargs)
ax.set(title='Mean', xlabel='', ylabel='', yscale='log', ylim=(7e1,5e5), yticks=(1e2,1e3,1e4,1e5))
baseline = stats2.loc[(stats2['group']=='marker') & (stats2['data_group']=='iPS11_transfection'), 'output_gmean'].mean()
ax.axhline(baseline, color='black', ls=':', zorder=0)

# std
ax = axes[1]
for construct, group in plot_df.groupby('construct'):
    sns.pointplot(data=group, x='ts_label', y='output_std', hue='construct', palette=app_palette,
                  ax=ax, marker=app_markers[construct], **scatter_kwargs)
ax.set(title='Std.', xlabel='', ylabel='', yscale='log', ylim=(1e4,1e6))

# slope
ax = axes[2]
for construct, group in plot_df.groupby('construct'):
    sns.pointplot(data=group, x='ts_label', y='slope', hue='construct', palette=app_palette,
                  ax=ax, marker=app_markers[construct], **scatter_kwargs)
ax.set(title='Slope', xlabel='', ylabel='',ylim=(0.5,1.3), yticks=[0.5,0.75,1,1.25], yticklabels=['0.5','','1.0',''])

fig.savefig(rd.outfile(output_path))

# perform statistical tests
f, axes = plt.subplots(1,3, figsize=(5,2))
pairs = [('base','CL'), ('OL','CL')]
stat_list = ['output_gmean', 'output_std', 'slope']
for ax, stat in zip(axes, stat_list):
    sns.stripplot(data=plot_df, x='ts_label', y=stat, ax=ax, hue='construct', palette=app_palette, legend=False,)
    if stat != 'slope': ax.set(yscale='log')
    annotator = Annotator(ax, pairs, data=plot_df, x='ts_label', y=stat,)
    annotator.configure(**annotate_kwargs, line_offset=2).apply_and_annotate()

In [ ]:
subfig = subfigures['B']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.4, bottom=0.3, top=0.6, right=0.15)
axes = subfig.subplots(1,3, gridspec_kw=dict(wspace=0.5))

plot_df = stats2[stats2['group'].isin(['controller','base']) & (stats2['design']<=1) & (stats2['promoter']=='CAG') & 
                 (stats2['data_group']=='iPS11_transfection')]

# mean
ax = axes[0]
for construct, group in plot_df.groupby('construct'):
    sns.pointplot(data=group, x='ts_label', y='output_gmean', hue='construct', palette=app_palette,
                  ax=ax, marker=app_markers[construct], **scatter_kwargs)
ax.set(title='Mean', xlabel='', ylabel='', yscale='log', ylim=(7e1,5e5), yticks=(1e2,1e3,1e4,1e5))
baseline = stats2.loc[(stats2['group']=='marker') & (stats2['data_group']=='iPS11_transfection'), 'output_gmean'].mean()
ax.axhline(baseline, color='black', ls=':', zorder=0)

# std
ax = axes[1]
for construct, group in plot_df.groupby('construct'):
    sns.pointplot(data=group, x='ts_label', y='output_std', hue='construct', palette=app_palette,
                  ax=ax, marker=app_markers[construct], **scatter_kwargs)
ax.set(title='Std.', xlabel='', ylabel='', yscale='log', ylim=(1e4,2e6))

# slope
ax = axes[2]
for construct, group in plot_df.groupby('construct'):
    sns.pointplot(data=group, x='ts_label', y='slope', hue='construct', palette=app_palette,
                  ax=ax, marker=app_markers[construct], **scatter_kwargs)
ax.set(title='Slope', xlabel='', ylabel='',ylim=(0.5,1.3), yticks=[0.5,0.75,1,1.25], yticklabels=['0.5','','1.0',''])

fig.savefig(rd.outfile(output_path))

# perform statistical tests
f, axes = plt.subplots(1,3, figsize=(5,2))
pairs = [('base','CL'), ('OL','CL')]
stat_list = ['output_gmean', 'output_std', 'slope']
for ax, stat in zip(axes, stat_list):
    sns.stripplot(data=plot_df, x='ts_label', y=stat, ax=ax, hue='construct', palette=app_palette, legend=False,)
    if stat != 'slope': ax.set(yscale='log')
    annotator = Annotator(ax, pairs, data=plot_df, x='ts_label', y=stat,)
    annotator.configure(**annotate_kwargs, line_offset=2).apply_and_annotate()

In [ ]:
subfig = subfigures['C']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.6, bottom=0.5, top=0.35, right=0.5)
axes = subfig.subplots(1,3, gridspec_kw=dict(width_ratios=(1,1,1), wspace=0.3))

# iPSC controls
group_order = ['base','miR','ts3','ts5']
plot_df = stats2[~stats2['group'].isin(['controller','marker']) & (stats2['promoter']=='EF1a') & (stats2['data_group']=='iPS11_transfection')].copy()
plot_df['group'] = plot_df['group'].astype(pd.api.types.CategoricalDtype(categories=group_order, ordered=True))
plot_df.sort_values(['group','ts'], inplace=True)

# shift xticks to add more space between promoter groups
buffer = 0.6
num_groups = 3
xtick_locs = [0, 1+buffer, 2+buffer] + [i+buffer*2 for i in range(3,7)]
construct_loc = {k:v for k,v in zip(plot_df['construct'].unique(), xtick_locs)}
plot_df['construct_loc'] = plot_df['construct'].replace(construct_loc)
metadata['construct_loc'] = metadata['construct'].map(construct_loc)
m = metadata.dropna()
m['construct_loc'] = m['construct_loc'].astype(str)
xlim = (-0.5, plot_df['construct_loc'].max()+0.5)

# adjust markers
m.loc[(m['miR_loc']=='CDS') | (m['ts_loc']=='3\''), 'markers'] = 'D'
m_dict = m.set_index('construct').to_dict('dict')
comb_markers = m_dict['markers']

# make xticklabels
def get_label(df):
    group = df['group'].unique()[0]
    d = df.copy()
    col_map = {'base': 'group', 'miR': 'miR', 'ts3': 'ts', 'ts5': 'ts'}
    d['label'] = d[col_map[group]]
    return d

m = m.groupby('group')[m.columns].apply(get_label).reset_index(drop=True)

# stat gmean
ax = axes[0]
for construct, group in plot_df.groupby('construct', sort=False):
    sns.pointplot(data=group, x='construct_loc', y='output_gmean', hue='construct', palette=main_palette,
                    ax=ax, marker=comb_markers[construct], **scatter_kwargs, native_scale=True)
ax.set(title='Mean', xlim=xlim, xlabel='', ylabel='', yscale='log', xticks=xtick_locs, ylim=(7e1,9e4))
baseline = stats2.loc[(stats2['group']=='marker') & (stats2['data_group']=='iPS11_transfection'), 'output_gmean'].mean()
ax.axhline(baseline, color='black', ls=':', zorder=0)
ax.minorticks_off()

# stat std
ax = axes[1]
for construct, group in plot_df.groupby('construct', sort=False):
    sns.pointplot(data=group, x='construct_loc', y='output_std', hue='construct', palette=main_palette,
                    ax=ax, marker=comb_markers[construct], **scatter_kwargs, native_scale=True)
ax.set(title='Standard deviation', xlim=xlim, xlabel='', ylabel='', yscale='log', xticks=xtick_locs, ylim=(1e4,1e6))

# slope
ax = axes[2]
for construct, group in plot_df.groupby('construct', sort=False):
    sns.pointplot(data=group, x='construct_loc', y='slope', hue='construct', palette=main_palette,
                    ax=ax, marker=comb_markers[construct], **scatter_kwargs, native_scale=True)
ax.set(title='Slope', xlim=xlim, xlabel='', ylim=(0.5,1.2), ylabel='', xticks=xtick_locs, 
       yticks=[0.5,0.6,0.7,0.8,0.9,1,1.1], yticklabels=['','0.6','','0.8','','1.0',''])

# add shaded region for miR-only constructs
for ax in axes:
    span1 = (xtick_locs[0]+(xtick_locs[1]-xtick_locs[0])/2, xtick_locs[2]+(xtick_locs[3]-xtick_locs[2])/2,)
    ax.axvspan(*span1, color=base.get_light_color(base.colors['gray']), alpha=0.2,)
    labels = dict(zip(m.drop_duplicates('construct_loc')['construct_loc'], m.drop_duplicates('construct_loc')['label']))
    ax.set_xticklabels([labels[l.get_text()].replace('.','-') for l in ax.get_xticklabels()], ha='right', rotation=45)
    ax.yaxis.set_minor_formatter(plt.NullFormatter())

fig.savefig(rd.outfile(output_path))

# perform statistical tests
f, axes = plt.subplots(1,3, figsize=(5,2))
construct_base = plot_df.loc[plot_df['group']=='base', 'construct'].values[0]
pairs = [(construct_base,c) for c in plot_df['construct'].unique() if c != construct_base]
stat_list = ['output_gmean', 'output_std', 'slope']
for ax, stat in zip(axes, stat_list):
    sns.stripplot(data=plot_df, x='construct', y=stat, ax=ax, hue='construct', palette=main_palette, legend=False,)
    labels = dict(zip(m.drop_duplicates('construct_loc')['construct'], m.drop_duplicates('construct_loc')['label']))
    ax.set_xticklabels([labels[l.get_text()].replace('.','-') for l in ax.get_xticklabels()], ha='right', rotation=45)
    if stat != 'slope': ax.set(yscale='log',)
    annotator = Annotator(ax, pairs, data=plot_df, x='construct', y=stat,)
    annotator.configure(**annotate_kwargs, line_offset=2).apply_and_annotate()

In [ ]:
subfig = subfigures['D']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.4, bottom=0.3, top=0.6, right=0.15)
axes = subfig.subplots(1,3, gridspec_kw=dict(wspace=0.5))

plot_df = stats2[(stats2['name'].str.contains('FXN')) & (stats2['data_group']=='therapeutic_transfection')]

# mean
ax = axes[0]
for construct, group in plot_df.groupby('construct'):
    sns.pointplot(data=group, x='ts_label', y='output_gmean', hue='construct', palette=app_palette,
                  ax=ax, marker=app_markers[construct], **scatter_kwargs)
ax.set(title='Mean', xlabel='', ylabel='', yscale='log', ylim=(2e1,4e5), yticks=(1e2,1e3,1e4,1e5))
baseline = stats2.loc[(stats2['group']=='marker') & (stats2['data_group']=='therapeutic_transfection'), 'output_gmean'].mean()
ax.axhline(baseline, color='black', ls=':', zorder=0)

# std
ax = axes[1]
for construct, group in plot_df.groupby('construct'):
    sns.pointplot(data=group, x='ts_label', y='output_std', hue='construct', palette=app_palette,
                  ax=ax, marker=app_markers[construct], **scatter_kwargs)
ax.set(title='Std.', xlabel='', ylabel='', yscale='log', ylim=(8e3,1.5e6))

# slope
ax = axes[2]
for construct, group in plot_df.groupby('construct'):
    sns.pointplot(data=group, x='ts_label', y='slope', hue='construct', palette=app_palette,
                  ax=ax, marker=app_markers[construct], **scatter_kwargs)
ax.set(title='Slope', xlabel='', ylabel='', ylim=(0.4,1.7), yticks=[0.5,0.75,1,1.25,1.5], yticklabels=['0.5','','1.0','','1.5'])

fig.savefig(rd.outfile(output_path))

# perform statistical tests
f, axes = plt.subplots(1,3, figsize=(5,2))
pairs = [('base','CL'), ('OL','CL')]
stat_list = ['output_gmean', 'output_std', 'slope']
for ax, stat in zip(axes, stat_list):
    sns.stripplot(data=plot_df, x='ts_label', y=stat, ax=ax, hue='construct', palette=app_palette, legend=False,)
    if stat != 'slope': ax.set(yscale='log')
    annotator = Annotator(ax, pairs, data=plot_df, x='ts_label', y=stat,)
    annotator.configure(**annotate_kwargs, line_offset=2).apply_and_annotate()

In [ ]:
subfig = subfigures['E']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.4, bottom=0.3, top=0.6, right=0.15)
axes = subfig.subplots(1,3, gridspec_kw=dict(wspace=0.5))

plot_df = stats2[(stats2['name'].str.contains('FMRP')) & (stats2['data_group']=='therapeutic_transfection')]

# mean
ax = axes[0]
for construct, group in plot_df.groupby('construct'):
    sns.pointplot(data=group, x='ts_label', y='output_gmean', hue='construct', palette=app_palette,
                  ax=ax, marker=app_markers[construct], **scatter_kwargs)
ax.set(title='Mean', xlabel='', ylabel='', yscale='log', ylim=(2e1,9e5), yticks=(1e2,1e3,1e4,1e5))
baseline = stats2.loc[(stats2['group']=='marker') & (stats2['data_group']=='therapeutic_transfection'), 'output_gmean'].mean()
ax.axhline(baseline, color='black', ls=':', zorder=0)

# std
ax = axes[1]
for construct, group in plot_df.groupby('construct'):
    sns.pointplot(data=group, x='ts_label', y='output_std', hue='construct', palette=app_palette,
                  ax=ax, marker=app_markers[construct], **scatter_kwargs)
ax.set(title='Std.', xlabel='', ylabel='', yscale='log', ylim=(1.5e3,3e5))

# slope
ax = axes[2]
for construct, group in plot_df.groupby('construct'):
    sns.pointplot(data=group, x='ts_label', y='slope', hue='construct', palette=app_palette,
                  ax=ax, marker=app_markers[construct], **scatter_kwargs)
ax.set(title='Slope', xlabel='', ylabel='', ylim=(0.6,1), yticks=[0.6,0.7,0.8,0.9,1], yticklabels=['0.6','','0.8','','1.0'])
    
fig.savefig(rd.outfile(output_path))

# perform statistical tests
f, axes = plt.subplots(1,3, figsize=(5,2))
pairs = [('base','CL'), ('OL','CL')]
stat_list = ['output_gmean', 'output_std', 'slope']
for ax, stat in zip(axes, stat_list):
    sns.stripplot(data=plot_df, x='ts_label', y=stat, ax=ax, hue='construct', palette=app_palette, legend=False,)
    if stat != 'slope': ax.set(yscale='log')
    annotator = Annotator(ax, pairs, data=plot_df, x='ts_label', y=stat,)
    annotator.configure(**annotate_kwargs, line_offset=2).apply_and_annotate()

In [ ]:
subfig = subfigures['F']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.5, bottom=0.4, top=0.35, right=0.15)
axes = subfig.subplots(1,4, gridspec_kw=dict(width_ratios=(1,1,1,1.5), wspace=0.5))

plot_df = stats[(stats['cell']=='therapeutic') & (stats['dox']==1000) & (stats['name'].str.contains('FXN'))]

# mean
ax = axes[0]
for construct, group in plot_df.groupby('construct'):
    sns.pointplot(data=group, x='ts_label', y='output_gmean', hue='construct', palette=app_palette,
                  ax=ax, marker=app_markers[construct], **scatter_kwargs)
ax.set(title='Mean', xlabel='', ylabel='', yscale='log', ylim=(3e1,7e3))
ax.set_xticklabels(ax.get_xticklabels(), ha='right', rotation=45)
baseline = baseline_df.loc[baseline_df['cell']=='therapeutic', 'output_gmean'].mean()
ax.axhline(baseline, color='black', ls=':', zorder=0)

# std
ax = axes[1]
for construct, group in plot_df.groupby('construct'):
    sns.pointplot(data=group, x='ts_label', y='output_std', hue='construct', palette=app_palette,
                  ax=ax, marker=app_markers[construct], **scatter_kwargs)
ax.set(title='Std.', xlabel='', ylabel='', yscale='log', ylim=(1e2,1.5e4))
ax.set_xticklabels(ax.get_xticklabels(), ha='right', rotation=45)

# slope
ax = axes[2]
for construct, group in plot_df.groupby('construct'):
    sns.pointplot(data=group, x='ts_label', y='slope', hue='construct', palette=app_palette,
                  ax=ax, marker=app_markers[construct], **scatter_kwargs)
ax.set(title='Slope', xlabel='', ylabel='', ylim=(0.2,1.2), yticks=[0.25,0.5,0.75,1], yticklabels=['','0.5','','1.0'])
ax.set_xticklabels(ax.get_xticklabels(), ha='right', rotation=45)

# CDF
ax = axes[3]
plot_df2 = quantiles[(quantiles['cell']=='therapeutic') & (quantiles['dox']==1000) & 
                    (quantiles['name'].str.contains('FXN')) & (quantiles['biorep']==1)]
plot_df2.sort_values(['ts_num','ts_label'], ascending=False, inplace=True)
plot_order = reversed(plot_df2['construct'].unique())

sns.kdeplot(data=plot_df2, x='output', hue='construct', palette=app_palette, ax=ax,
            cumulative=True, common_norm=False, legend=False, log_scale=True)
ax.set(xticks=np.logspace(2,6,5), ylabel='', xlim=(1e1,2e4), title='CDF')
ax.minorticks_off()
ax.grid(zorder=-1, color=base.get_light_color(base.get_light_color(base.colors['gray'])), which='both',alpha=0.7)

fig.savefig(rd.outfile(output_path))

# perform statistical tests
f, axes = plt.subplots(1,3, figsize=(5,2))
pairs = [('base','CL'), ('OL','CL')]
stat_list = ['output_gmean', 'output_std', 'slope']
for ax, stat in zip(axes, stat_list):
    sns.stripplot(data=plot_df, x='ts_label', y=stat, ax=ax, hue='construct', palette=app_palette, legend=False,)
    if stat != 'slope': ax.set(yscale='log')
    annotator = Annotator(ax, pairs, data=plot_df, x='ts_label', y=stat,)
    annotator.configure(**annotate_kwargs, line_offset=2).apply_and_annotate()

In [ ]:
subfig = subfigures['G']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.3, bottom=0.4, top=0.35, right=0.15)
axes = subfig.subplots(1,1,)

plot_df = stats[(stats['cell']=='therapeutic') & (stats['dox']==0) & (stats['name'].str.contains('FXN'))]

# stat gmean
ax = axes 
for construct, group in plot_df.groupby('construct'):
    sns.pointplot(data=group, x='ts_label', y='output_gmean', hue='construct', palette=app_palette,
                    ax=ax, marker=app_markers[construct], **scatter_kwargs)
ax.set(title='Mean', xlabel='', ylabel='', yscale='log', ylim=(1e1,2e2))
baseline = baseline_df.loc[(baseline_df['cell']=='iPS11') & (baseline_df['dox']==0), 'output_gmean'].mean()
ax.axhline(baseline, color='black', ls=':')
ax.set_xticklabels(ax.get_xticklabels(), ha='right', rotation=45)

fig.savefig(rd.outfile(output_path))

In [ ]:
subfig = subfigures['H']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.5, bottom=0.4, top=0.35, right=0.15)
axes = subfig.subplots(1,4, gridspec_kw=dict(width_ratios=(1,1,1,1.5), wspace=0.5))

plot_df = stats[(stats['cell']=='therapeutic') & (stats['dox']==1000) & (stats['name'].str.contains('FMRP'))]

# mean
ax = axes[0]
for construct, group in plot_df.groupby('construct'):
    sns.pointplot(data=group, x='ts_label', y='output_gmean', hue='construct', palette=app_palette,
                  ax=ax, marker=app_markers[construct], **scatter_kwargs)
ax.set(title='Mean', xlabel='', ylabel='', yscale='log', ylim=(3e1,9e4))
ax.set_xticklabels(ax.get_xticklabels(), ha='right', rotation=45)
baseline = baseline_df.loc[baseline_df['cell']=='therapeutic', 'output_gmean'].mean()
ax.axhline(baseline, color='black', ls=':', zorder=0)

# std
ax = axes[1]
for construct, group in plot_df.groupby('construct'):
    sns.pointplot(data=group, x='ts_label', y='output_std', hue='construct', palette=app_palette,
                  ax=ax, marker=app_markers[construct], **scatter_kwargs)
ax.set(title='Std.', xlabel='', ylabel='', yscale='log', ylim=(1e3,5e4))
ax.set_xticklabels(ax.get_xticklabels(), ha='right', rotation=45)

# slope
ax = axes[2]
for construct, group in plot_df.groupby('construct'):
    sns.pointplot(data=group, x='ts_label', y='slope', hue='construct', palette=app_palette,
                  ax=ax, marker=app_markers[construct], **scatter_kwargs)
ax.set(title='Slope', xlabel='', ylabel='', ylim=(0.1,0.75), yticks=[0.2,0.4,0.6],)
ax.set_xticklabels(ax.get_xticklabels(), ha='right', rotation=45)

# CDF
ax = axes[3]
plot_df2 = quantiles[(quantiles['cell']=='therapeutic') & (quantiles['dox']==1000) & 
                    (quantiles['name'].str.contains('FMRP')) & (quantiles['biorep']==1)]
plot_df2.sort_values(['ts_num','ts_label'], ascending=False, inplace=True)
plot_order = reversed(plot_df2['construct'].unique())

sns.kdeplot(data=plot_df2, x='output', hue='construct', palette=app_palette, ax=ax,
            cumulative=True, common_norm=False, legend=False, log_scale=True)
ax.set(xticks=np.logspace(2,6,5), ylabel='', xlim=(1e2,1e5), title='CDF')
ax.minorticks_off()
ax.grid(zorder=-1, color=base.get_light_color(base.get_light_color(base.colors['gray'])), which='both',alpha=0.7)

fig.savefig(rd.outfile(output_path))

# perform statistical tests
f, axes = plt.subplots(1,3, figsize=(5,2))
pairs = [('base','CL'), ('OL','CL')]
stat_list = ['output_gmean', 'output_std', 'slope']
for ax, stat in zip(axes, stat_list):
    sns.stripplot(data=plot_df, x='ts_label', y=stat, ax=ax, hue='construct', palette=app_palette, legend=False,)
    if stat != 'slope': ax.set(yscale='log')
    annotator = Annotator(ax, pairs, data=plot_df, x='ts_label', y=stat,)
    annotator.configure(**annotate_kwargs, line_offset=2).apply_and_annotate()

In [ ]:
subfig = subfigures['I']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.3, bottom=0.4, top=0.35, right=0.15)
axes = subfig.subplots(1,1,)

plot_df = stats[(stats['cell']=='therapeutic') & (stats['dox']==0) & (stats['name'].str.contains('FMRP'))]

# stat gmean
ax = axes #[0]
for construct, group in plot_df.groupby('construct'):
    sns.pointplot(data=group, x='ts_label', y='output_gmean', hue='construct', palette=app_palette,
                    ax=ax, marker=app_markers[construct], **scatter_kwargs)
ax.set(title='Mean', xlabel='', ylabel='', yscale='log', ylim=(1e1,2e2))
baseline = baseline_df.loc[(baseline_df['cell']=='iPS11') & (baseline_df['dox']==0), 'output_gmean'].mean()
ax.axhline(baseline, color='black', ls=':')
ax.set_xticklabels(ax.get_xticklabels(), ha='right', rotation=45)

fig.savefig(rd.outfile(output_path))

In [ ]:
subfig = subfigures['K']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.45, bottom=0.4, top=0.35, right=0.15)
axes = subfig.subplots(1,2, sharey=True, gridspec_kw=dict(wspace=0.3, width_ratios=(1,0.75)))

text_color = base.colors['orange']

# FXN
ax = axes[0]
plot_df = stats_qpcr[(stats_qpcr['primers']=='FXN') & ~(stats_qpcr['construct'].isin(['no-RT','no-cDNA']))]

for construct, group in plot_df.groupby('construct'):
    sns.pointplot(data=group, x='ts_label', y='relative_expression', ax=ax, hue='construct', palette=app_palette,
                    marker=main_markers[construct], **scatter_kwargs)
ax.set(xlabel='', ylim=(-0.1,2), ylabel='Expression, $2^{-\Delta\Delta Ct}$')
ax.set_title('FXN', style='italic', color=text_color)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')

ui_value = plot_df.loc[plot_df['construct']=='UI', 'relative_expression'].mean()
ax.annotate(f'{1/ui_value:0.1f}', (0.05,1), xycoords='axes fraction', fontsize=base.font_sizes['smaller_size'], ha='left', va='top')
ui_value_to_CL = (plot_df.loc[plot_df['ts_label']=='CL', 'relative_expression'].mean()) / ui_value
ax.annotate(f'{ui_value_to_CL:0.2f}', (2,1), xycoords=('data','axes fraction'), fontsize=base.font_sizes['smaller_size'], 
            ha='center', va='top', color=text_color)

# mRuby2-1
ax = axes[1]
plot_df = stats_qpcr[(stats_qpcr['primers']=='mRuby2-1') & ~(stats_qpcr['construct'].isin(['no-RT','no-cDNA','UI']))]

for construct, group in plot_df.groupby('construct'):
    sns.pointplot(data=group, x='ts_label', y='relative_expression', ax=ax, hue='construct', palette=app_palette,
                    marker=main_markers[construct], **scatter_kwargs)
ax.set(xlabel='', ylim=(-0.1,2), ylabel='')
ax.set_title('mRuby2', style='italic', color=text_color)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')

for ax in axes:
    ax.axhline(1, c='black', ls=':', zorder=-10)
    ax.axhline(0, c='black', ls=':', zorder=-10)

fig.savefig(rd.outfile(output_path))

In [ ]:
subfig = subfigures['L']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.45, bottom=0.4, top=0.35, right=0.15)
axes = subfig.subplots(1,2, sharey=True, gridspec_kw=dict(wspace=0.3))

text_color = base.colors['red']

# FMRP-2
ax = axes[0]
plot_df = stats_qpcr[(stats_qpcr['primers']=='FMRP-2') & ~(stats_qpcr['construct'].isin(['no-RT','no-cDNA','UI']))]

for construct, group in plot_df.groupby('construct'):
    sns.pointplot(data=group, x='ts_label', y='relative_expression', ax=ax, hue='construct', palette=app_palette,
                    marker=main_markers[construct], **scatter_kwargs)
ax.set(xlabel='', ylim=(-0.1,1.4), ylabel='Expression, $2^{-\Delta\Delta Ct}$')
ax.set_title('Fmr1', style='italic', color=text_color)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')

# EGFP-2
ax = axes[1]
plot_df = stats_qpcr[(stats_qpcr['primers']=='EGFP-2') & ~(stats_qpcr['construct'].isin(['no-RT','no-cDNA','UI']))]

for construct, group in plot_df.groupby('construct'):
    sns.pointplot(data=group, x='ts_label', y='relative_expression', ax=ax, hue='construct', palette=app_palette,
                    marker=main_markers[construct], **scatter_kwargs)
ax.set(xlabel='', ylim=(-0.1,2.5), ylabel='')
ax.set_title('EGFP', style='italic', color=text_color)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')

for ax in axes:
    ax.axhline(1, c='black', ls=':', zorder=-10)
    ax.axhline(0, c='black', ls=':', zorder=-10)

fig.savefig(rd.outfile(output_path))

In [ ]:
# Save to OneDrive
fig.savefig(rd.outfile(rd.datadir/'manuscripts'/'2024_miR-iFFL'/'figures'/'links'/'fig_applications-supp.pdf'))

In [ ]:
# perform statistical tests
plots = {
    'FXN': ['FXN','mRuby2-1','mRuby2-2'],
    'FMRP': ['FMRP-1','FMRP-2','EGFP-1','EGFP-2']
}
pairs = [('base','OL'), ('base','CL'), ('OL','CL'), ('none','CL')]
y = 'relative_expression'
stats_qpcr[y] = stats_qpcr[y].astype(float)
ylabel = '$2^{-\Delta\Delta C_t}$'

for name, primer_list in plots.items():
    f, axes = plt.subplots(1,len(primer_list), figsize=(10,5), sharey=True)

    for ax, primers in zip(axes, primer_list):
        plot_df = stats_qpcr[((stats_qpcr['name'].str.contains(name)) | (stats_qpcr['construct']=='UI')) & (stats_qpcr['primers']==primers)]
        sns.stripplot(data=plot_df, x='ts_label', y=y, hue='biorep', ax=ax, legend=False)
        ax.set(xlabel='', ylabel='', title=primers)
        sns.despine(ax=ax)
        annotator = Annotator(ax, pairs, data=plot_df, x='ts_label', y=y,)
        annotator.configure(test='t-test_ind', text_format='star', loc='inside', line_height=0,
                            text_offset=-2, line_offset_to_group=10).apply_and_annotate()
        ax.axhline(0, color='black', zorder=0)
    axes[0].set(ylabel=ylabel,)

In [ ]:
result = stats_qpcr[stats_qpcr['construct']=='UI'].groupby('primers')[['relative_expression']].mean()
display(result)